In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import math
import graph_module as gm

In [ ]:
# dtype 다양한 변경방법
'''
df['Dates'] = pd.to_datetime(df['Dates'], format='%y%m%d') 
df['Date'] = df['Date'].astype('datetime64[ns]')
dtype = pd.SparseDtype(np.dtype('datetime64[ns]'))
series = pd.Series(df.date, dtype=dtype)
df['date']=np.array(series)
'''

In [ ]:
read_excel_path = 'C:/jimi2cool/pqr_project/input/clean/'
write_pickle_path = 'C:/jimi2cool/pqr_project/input/clean_pickle/'

In [ ]:
# read excel data
parser = lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce')

act_loan_outstanding = pd.read_excel(read_excel_path+'TR2_20210602.xlsx', date_parser=parser, parse_dates=['실행일자', '여신시작일', '여신만기일', '해지일자', '마감일자'])
monthly_loan_outstanding = pd.read_excel(read_excel_path+'여신잔액_final_20210531.xlsx', dtype ={'기준월' : str, '기준일자' : str})
pre_payment = pd.read_excel(read_excel_path+'TR2608_중도상환_20210606.xlsx')
monthly_delinquent = pd.read_excel(read_excel_path+'월말기준30일이상연체2016_202105.xlsx')
lastMonth_loan_outstanding = pd.read_excel(read_excel_path+'TR2818_20210531.xlsx', date_parser=parser, parse_dates=['최초수납일']) # TR2818

In [ ]:
# export picke data to pickle 폴더
act_loan_outstanding.to_pickle(write_pickle_path + 'act_loan_outstanding.pkl')
monthly_loan_outstanding.to_pickle(write_pickle_path + 'monthly_loan_outstanding.pkl')
pre_payment.to_pickle(write_pickle_path + 'pre_payment.pkl')
monthly_delinquent.to_pickle(write_pickle_path + 'monthly_delinquent.pkl')
lastMonth_loan_outstanding.to_pickle(write_pickle_path + 'lastMonth_loan_outstanding.pkl')

In [29]:
def brand_cat(brand) :
    if brand in main_car_brand :
        return brand
    else :
        return "etc. Brand"

In [34]:
def product_cat(product) :
    if product in main_product :
        if product in ['신차운용리스'] : 
            return '오토리스'
        elif product in ['신차금융리스', '신차할부금융', '오토론신차대출'] :
            return '할부/오토론'
        elif product in ['스마트스토어대출', '스탁론(국내)', '임차보증금대출'] :
            return product
        else :
            return product
    else :
        if product in ['오토론중고차대출', '바이크운용리스'] :
            return '오토_기타'
        elif product in ['스탁론(해외)', '생명연계대출', '미트론', '주식담보대출' ,'NPL대출'] :
            return "데이터_기타"
        else : 
            return product

In [36]:
# TR2 데이터 정비
act_loan_outstanding.columns = act_loan_outstanding.columns.str.replace("\n", "")
analysis_cols = [
    '부서', '상품명', '실행번호', '실행상태', '고객구분', '고객명', 
    '고객번호',  '실행일자', '실행사유', '여신금액', '여신잔액', '여신시작일', '여신만기일', '여신기간',  '고객이자율', 'IRR', 
    '차량가격', '보증금', '보증보험금액',  '무보증잔가', '보증잔가', '원금유예금액', '선수금', '장기선수금', '표준산업분류코드', 
    '차량코드',  '제조사', '시리즈', '물건명', '차량번호', '차대번호', '유종', '잔가보장업체', '잔가업체보장금액', '초과잔가율(%)', 
    '프로모션(%)', '감가면제여부(YN)', '이전실행번호', '최초실행번호', '해지일자', '마감일자', '마감사유', '해지사유',  '공급자', 
    '공급사제휴여부', '딜러', '딜러지점',  '에이전트', '금융담당자'
    ]
org_digital = ['데이터금융1팀', '데이터금융2팀', '데이터금융3팀'] # 데이터금융본부
org_investment = ['투자금융1팀', '투자금융2팀', '투자금융3팀'] # 데이터금융본부

product_auto = ['신차운용리스', '신차금융리스', '신차할부금융', '오토론신차대출', '오토론중고차대출', '바이크운용리스'] # 오토상품
product_digital = ['스마트스토어대출', '스탁론(국내)', '스탁론(해외)', '임차보증금대출', '생명연계대출', '미트론', '주식담보대출' ,'NPL대출'] # 디지털금융본부 상품

filter_product = ['신차운용리스', '신차금융리스', '신차할부금융', '오토론신차대출', '오토론중고차대출', '바이크운용리스', '스마트스토어대출', '스탁론(국내)', '스탁론(해외)', '임차보증금대출', '생명연계대출', '미트론', '주식담보대출' ,'NPL대출']

# 주요 차량 브랜드
main_car_brand = ['MERCEDES BENZ', 'BMW', 'AUDI', 'PORSCHE']

# 리테일 주요 상품
main_product = ['신차운용리스', '신차금융리스', '신차할부금융', '오토론신차대출', '스마트스토어대출', '스탁론(국내)', '임차보증금대출' ]
# 리테일 sub 상품
sub_product = ['오토론중고차대출', '바이크운용리스', '스탁론(해외)', '생명연계대출', '미트론', '주식담보대출' ,'NPL대출']

# 오토리스 분석제외 기업(도이치, 쏘카, SK네트웍스, 딜러사 금융 등)
exception_company = [1008109, 1000518, 1003192, 1029989, 1001717, 1003580]

# 예외기업 제외
# df = df[~df['고객번호'].isin(exception_company)]

#결측값 가진 열의 행 삭제
null_list = list(act_loan_outstanding[act_loan_outstanding['실행번호'].isnull()].index)
act_loan_outstanding.drop(null_list, axis = 0, inplace = True)

# 실행상태 임시 데이터 삭제
act_loan_outstanding = act_loan_outstanding[act_loan_outstanding['실행상태'] != '임시']

act_loan_outstanding_filtered = act_loan_outstanding.filter(items = analysis_cols)
lastMonth_loan_outstanding_filtered = lastMonth_loan_outstanding[['실행번호', '최초수납일']]

In [ ]:
clientzip_df = pd.merge(df, df1, right_on = '실행번호', left_on = '실행번호')

In [ ]:
data_loan_outstanding = pd.merge(act_loan_outstanding_filtered, lastMonth_loan_outstanding_filtered, right_on = '실행번호', left_on = '실행번호')

In [40]:
data_loan_outstanding['closing_mon'] = data_loan_outstanding['마감일자'].dt.strftime('%y-%m')
data_loan_outstanding['vin_yob'] = data_loan_outstanding.loc[:,'실행일자'].dt.strftime('%y')
data_loan_outstanding['vin_mob'] = data_loan_outstanding.loc[:,'실행일자'].dt.strftime('%y-%m')
data_loan_outstanding['vin_qob'] = data_loan_outstanding['실행일자'].dt.year.astype(str) + 'Q' + data_loan_outstanding['실행일자'].dt.quarter.astype(str)
data_loan_outstanding['pre-payed_duration'] = (data_loan_outstanding['마감일자'] - data_loan_outstanding['실행일자']).dt.days

In [41]:
data_loan_outstanding['car_main_brand'] = data_loan_outstanding['제조사'].apply(brand_cat)

In [42]:
data_loan_outstanding['상품분류'] = data_loan_outstanding['상품명'].apply(product_cat)

In [43]:
data_loan_outstanding['오토특수법인'] = data_loan_outstanding['고객번호'].apply(lambda x : '오토특수' if x in exception_company else '오토일반')

In [45]:
data_loan_outstanding['본인부담율'] = ((data_loan_outstanding['보증금'] + data_loan_outstanding['장기선수금'] + data_loan_outstanding['선수금'])/data_loan_outstanding['차량가격'])*100

In [46]:
data_loan_outstanding['잔가율'] = (data_loan_outstanding['무보증잔가'] / data_loan_outstanding['차량가격'])*100
data_loan_outstanding['return_mob'] = data_loan_outstanding.loc[:,'여신만기일'].dt.strftime('%y-%m')

In [ ]:
# monthlyProduct_cross_counts = pd.crosstab(df5['vin_mob'], df5['상품명'], df5['실행번호'], aggfunc='count')
# monthlyProduct_cross_sum = pd.crosstab(df5['vin_mob'], df5['상품명'], df5['여신금액']/100000000, aggfunc='sum')
# monthlyProduct_cross_average = pd.crosstab(df5['vin_mob'], df5['상품명'], df5['여신금액']/1000000, aggfunc='mean')
# df6 = df5[df5['상품명'] =="신차운용리스"]
# operaing_lease_counts = pd.crosstab(df6['vin_mob'], df6['제조사'], df6['실행번호'], aggfunc='count')

# monthly_return = pd.crosstab(oper_lease_df['vin_mob'], oper_lease_df['제조사'], oper_lease_df['실행번호'], aggfunc='count')
# monthly_return_amt = pd.crosstab(oper_lease_df['return_mob'], oper_lease_df['제조사'], oper_lease_df['무보증잔가']/100000000, aggfunc='sum')

# closing_stat = pd.crosstab(auto_closing_stat['closing_mon'], auto_closing_stat['마감사유'], auto_closing_stat['실행번호'], aggfunc='count')
# due_closing_stat = pd.crosstab(due_closing_df['closing_mon'], due_closing_df['마감사유'], due_closing_df['실행번호'], aggfunc='count')
# monthlyProduct_entitytype_counts = pd.crosstab(df5['vin_mob'], df5['고객구분'], df5['실행번호'], aggfunc='count')


In [ ]:
#pdf 생성
mypdf = PdfPages('pqr_chart_20210506.pdf')
# 파라미터 설정
plt.rcParams.update({'font.size': 10})
plt.rcParams.update({'font.family': 'Gulim'})
